In [ ]:
import os
import cv2
import itertools as it
from tqdm import tqdm_notebook as tqdm
import numpy as np 
from matplotlib import pyplot as plt


from impl import *

In [ ]:
images_dir = "E:/Bobst/Images"
images_paths = ["{d}/{f}".format(d=images_dir, f=f) for f in os.listdir(images_dir)]

In [ ]:
images = [cv2.imread(p, 0) for p in tqdm(images_paths[1650:1660])]

In [ ]:
template = cv2.imread(images_paths[0], 0)

In [ ]:
def_pos = [np.array([320, 1098]), np.array([1350, 1097])]
templates = [template[1100:1225, 320:440], template[1100:1225, 1350:1650]]
    

In [ ]:
template_img = cv2.imread(images_paths[0], 0)
def_pos = np.array([320, 1098])
def_pos2 = np.array([1350, 1097])
template_1 = template_img[1100:1225, 320:440]
template_3 = template_img[1100:1225, 1350:1650]

In [ ]:
def old_align(img, template):
    
    # Find size of image1
    sz = template.shape

    # Define the motion model
    warp_mode = cv2.MOTION_AFFINE
    warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Run the ECC algorithm. The results are stored in warp_matrix.
    (_, warp_matrix) = cv2.findTransformECC(template, img, warp_matrix, warp_mode)

    img_aligned = cv2.warpAffine(img, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    return img_aligned

In [ ]:
aligned_images = [sub_align(img) for img in tqdm(images)]

In [ ]:
aligned_images = [old_align(img, images[0]) for img in tqdm(images)]

In [ ]:
std = np.std(aligned_images, axis=0)
std.shape

In [ ]:
mean = np.mean(aligned_images, axis=0)
mean.shape

In [ ]:
dif = mean - aligned_images[7]

In [ ]:
dif_contrast = contrast(dif, 15)

In [ ]:
np.max(dif_contrast)

In [ ]:
save(dif_contrast, "dif_contrast.jpg") 

In [ ]:
show(dif_contrast) 

In [ ]:
show(dif_contrast) 

In [ ]:
cv2.imwrite("E:\dif_p.jpg", (dif > 0) * dif)

In [ ]:
dif.shape, dif[0, 0]

In [ ]:
dif_constrast = (dif > 50) * 255 # * dif

In [ ]:
(dif_constrast > 0).sum()

In [ ]:
cv2.imwrite("E:\dif_p_contrast.jpg", dif_constrast)

In [ ]:
np.nonzero(std - std2)

In [ ]:
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

In [ ]:
kp_des = [orb.detectAndCompute(img,None) for img in tqdm(aligned_images)]

In [ ]:
combinations = list(it.combinations(range(len(kp_des)), 2))

In [ ]:
all_matches = [bf.match(kp_des[i][1], kp_des[j][1]) for i,j in tqdm(combinations)]

In [ ]:
distances = [np.sum([m.distance for m in matches]) for matches in all_matches]

In [ ]:
np.argsort(distances)

In [ ]:
np.max(distances), np.min(distances)

In [ ]:
idx = np.argsort(distances)[0]
distances[idx], combinations[idx]

In [ ]:
img_idx1 = combinations[idx][0]
img_idx2 = combinations[idx][1]

kp1 = kp_des[img_idx1][0]
kp2 = kp_des[img_idx2][0]

matches = all_matches[idx]

img1 = aligned_images[img_idx1]
img2 = aligned_images[img_idx2]

In [ ]:
img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches, None, flags=2)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(img3)
plt.show()

In [ ]:
dif = cv2.subtract(img1, img2)

In [ ]:
dif2 = cv2.subtract(resized_img[0], resized_img[1])

In [ ]:
cv2.imwrite("E:\dif2.jpg", dif2)

In [ ]:
plt.figure(figsize=(20, 15))
plt.imshow(dif2)
plt.show()

In [ ]:
img = cv2.imread(pictures[0])
img_cropped = crop_contour(img)

cv2.imwrite("E:\diff.jpg", img_cropped)

In [ ]:
mask = np.zeros_like(im) + 255 # Create mask where white is what we want, black otherwise
cv2.drawContours(mask, cnt, idx, 255, -1) # Draw filled contour in mask
out = np.zeros_like(im) # Extract out the object and place into output image
out[mask == 255] = im[mask == 255]

cv2.imwrite("E:\diff.jpg", out)

In [ ]:
def draw_contour(img):
    img = img.copy()
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    _, thresh = cv2.threshold(img_gray,127,255,0)
    
    _, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv2.contourArea)
    
    cv2.drawContours(img, [cnt], 0, (0,255,0), 3)
    return img

In [ ]:
img = cropped_images[1]#draw_contour(images[-4])

plt.figure(figsize=(20, 20))
plt.imshow(img)
plt.show()

In [ ]:
im1 =  cv2.imread(images_paths[0]);
im2 =  cv2.imread(images_paths[1]);

cropped1 = crop_contour(im1)
cropped2 = crop_contour(im2)

min_shape = get_min_width_height([cropped1, cropped2])

resized1 = cv2.resize(cropped1, min_shape)
resized2 = cv2.resize(cropped2, min_shape)

cv2.imwrite("E:\img_resized1.jpg", resized1)
cv2.imwrite("E:\img_resized2.jpg", resized2)

In [ ]:
# Read the images to be aligned
img2 =  cv2.imread(images_paths[2]);
 
img2_aligned = align(img)

cv2.imwrite("E:\img1.jpg", img_template)
cv2.imwrite("E:\img2.jpg", img2)
cv2.imwrite("E:\img_aligned.jpg", img2_aligned)

In [ ]:
def contour(img):
    
    _, thresh = cv2.threshold(img,127,255,0)
    
    _, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv2.contourArea)
    
    return cnt

In [ ]:

def show(img):
    plt.figure(figsize=(20, 20))
    plt.imshow(img, cmap='gray', vmin = 0, vmax = 255)
    plt.show()

In [ ]:
def sub_align(img):
    rows,cols = img.shape
    res_1_ = cv2.matchTemplate(img,template_1,cv2.TM_CCOEFF)
    res_3_ = cv2.matchTemplate(img,template_3,cv2.TM_CCOEFF)

    _, _, _, max_loc_1_ = cv2.minMaxLoc(res_1_)
    _, _, _, max_loc_3_ = cv2.minMaxLoc(res_3_)
    del res_1_
    del res_3_
    pos_1 = np.array([max_loc_1_[0], max_loc_1_[1]])
    pos_3 = np.array([max_loc_3_[0], max_loc_3_[1]])
    
    #translate
    trans = def_pos - pos_1
    M = np.float32([[1,0,trans[0]],[0,1,trans[1]]])
    dst = cv2.warpAffine(img,M,(cols,rows))
    #rotation
    angle = angle_between(pos_3-pos_1, def_pos2-def_pos)
    M = cv2.getRotationMatrix2D((def_pos[0],def_pos[1]),angle,1)
    dst = cv2.warpAffine(dst,M,(cols,rows))
    return dst
def aligne(img, path):
    template_img = cv2.imread(path, 0)
    def_pos = np.array([320, 1098])
    def_pos2 = np.array([1350, 1097])
    template_1 = template_img[1100:1225, 320:440]
    template_3 = template_img[1100:1225, 1350:1650]
    return sub_align(img)

In [ ]:
def save(img, filename):
    return cv2.imwrite("E:/Bobst/{}".format(filename), img)